In [1]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
import psycopg2
from getpass import getpass

In [2]:
password2 = getpass('database password')

In [6]:
import pandas.io.sql as psql
connection = psycopg2.connect(
# make sure you use your local database name, user and host (should be localhost by default)
            host="localhost",
            database="final-project",
            user="postgres",
            password=password2)
df = psql.read_sql('SELECT * FROM survey_data', connection)

In [7]:
df = df[df.cps19_votechoice != "Don't know/ Prefer not to answer"]
df = df[df.cps19_votechoice != "Another party (please specify)"]
df['cps19_votechoice'].replace({'Bloc Qu<e9>b<e9>cois':'Bloc Quebecois'}, inplace=True)
df.head(5)

,cps19_responseid,cps19_citizenship,cps19_yob,cps19_gender,cps19_province,cps19_education,cps19_demsat,cps19_interest_gen_1,cps19_interest_elxn_1,cps19_v_likely,...,cps19_language_vietnamese,cps19_language_no_answer,cps19_language_aborginal,cps_19_language_other,cps19_employment,cps19_union,cps19_children,cps19_income_number,cps19_marital,cps19_household
1,R_3j7fAVYfVCewi3H,Canadian citizen,2000,A woman,Ontario,Some university,Fairly satisfied,10.0,10.0,Certain to vote,...,No,No,No,No,Student and working for pay,No,No,30000.0,Never Married,5
2,R_brdMqsPTvQ5t1tL,Canadian citizen,2000,A woman,Ontario,Completed secondary/ high school,Fairly satisfied,8.0,8.0,Certain to vote,...,No,No,No,No,Working for pay part-time,No,No,13000.0,Never Married,2
3,R_Wumhl7QEMURFqZH,Canadian citizen,2000,A man,Ontario,Completed secondary/ high school,Fairly satisfied,9.0,8.0,Certain to vote,...,No,No,No,No,Student and working for pay,No,No,55000.0,Never Married,3
4,R_3EH051N9vLmOOHM,Canadian citizen,2000,A woman,Ontario,Completed secondary/ high school,Fairly satisfied,10.0,10.0,Certain to vote,...,No,No,No,No,Student,No,No,190000.0,Never Married,4
5,R_1FynCZH7i2j3zEY,Canadian citizen,1999,A man,Ontario,Completed secondary/ high school,Fairly satisfied,10.0,10.0,Certain to vote,...,No,No,No,Hebrew,Working for pay part-time,No,No,10000.0,Never Married,2


In [8]:
df.columns

Index(['cps19_responseid', 'cps19_citizenship', 'cps19_yob', 'cps19_gender',
       'cps19_province', 'cps19_education', 'cps19_demsat',
       'cps19_interest_gen_1', 'cps19_interest_elxn_1', 'cps19_v_likely',
       'cps19_votechoice', 'cps19_fed_gov_sat', 'cps19_lr_scale',
       'cps19_spend_educ', 'cps19_spend_env', 'cps19_spend_just_law',
       'cps19_spend_defence', 'cps19_spend_imm_min', 'cps19_econ_retro',
       'cps19_econ_fed', 'cps19_ownfinanc_fed', 'cps19_imm', 'cps19_refugees',
       'cps19_govt_confusing', 'cps19_govt_say', 'cps19_pol_eth',
       'cps19_lib_promises', 'cps19_snclav', 'cps19_volunteer',
       'cps19_duty_choice', 'cps19_own_fin_retro', 'cps19_prov_gov_sat',
       'cps19_groups_therm_1', 'cps19_groups_therm_2', 'cps19_groups_therm_3',
       'cps19_groups_therm_4', 'cps19_groups_therm_5', 'cps19_turnout_2015',
       'cps19_religion', 'cps19_bornin_canada', 'cps19_ethnicity_first_nat',
       'cps19_ethnicity_british', 'cps19_ethnicity_chinese',
    

In [9]:
columns = list(df.columns)
columns
target = ['cps19_votechoice']

In [10]:
df.describe()

,cps19_yob,cps19_interest_gen_1,cps19_interest_elxn_1,cps19_lr_scale,cps19_groups_therm_1,cps19_groups_therm_2,cps19_groups_therm_3,cps19_groups_therm_4,cps19_groups_therm_5,cps19_income_number,cps19_household
count,11812.000000,11812.000000,11812.000000,11812.000000,11812.000000,11812.000000,11812.000000,11812.000000,11812.000000,1.181200e+04,11812.000000
mean,1969.287081,7.171267,7.734761,5.168981,65.023366,62.614037,62.793684,58.468676,45.549611,9.137942e+04,12.868270
std,16.282712,2.115185,2.050914,2.522555,25.704717,26.463899,27.725001,28.007775,23.530121,1.652481e+05,806.501689
min,1920.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1.000000
25%,1956.000000,6.000000,7.000000,3.000000,51.000000,50.000000,48.000000,40.000000,30.000000,4.600000e+04,2.000000
50%,1969.000000,7.000000,8.000000,5.000000,70.000000,66.000000,65.000000,61.000000,47.000000,7.500000e+04,2.000000
75%,1983.000000,9.000000,9.000000,7.000000,84.000000,82.000000,85.000000,80.000000,62.000000,1.100000e+05,3.000000
max,2001.000000,10.000000,10.000000,10.000000,100.000000,100.000000,100.000000,100.000000,100.000000,7.696588e+06,72000.000000


In [31]:
X = df.drop(['cps19_responseid', 'cps19_votechoice', 'cps19_ethnicity_no_answer','cps19_ethnicity_other_1', 'cps19_ethnicity_other_2', 'cps19_language_no_answer', 'cps_19_language_other'],axis=1)
X = pd.get_dummies(X, columns=['cps19_citizenship', 'cps19_yob',
       'cps19_gender', 'cps19_province', 'cps19_education', 'cps19_demsat',
       'cps19_interest_gen_1', 'cps19_interest_elxn_1', 'cps19_v_likely', 'cps19_fed_gov_sat', 'cps19_lr_scale',
       'cps19_spend_educ', 'cps19_spend_env', 'cps19_spend_just_law',
       'cps19_spend_defence', 'cps19_spend_imm_min', 'cps19_econ_retro',
       'cps19_econ_fed', 'cps19_ownfinanc_fed', 'cps19_imm', 'cps19_refugees',
       'cps19_govt_confusing', 'cps19_govt_say', 'cps19_pol_eth',
       'cps19_lib_promises', 'cps19_snclav', 'cps19_volunteer',
       'cps19_duty_choice', 'cps19_own_fin_retro', 'cps19_prov_gov_sat',
       'cps19_groups_therm_1', 'cps19_groups_therm_2', 'cps19_groups_therm_3',
       'cps19_groups_therm_4', 'cps19_groups_therm_5', 'cps19_turnout_2015',
       'cps19_religion', 'cps19_bornin_canada', 'cps19_ethnicity_first_nat',
       'cps19_ethnicity_british', 'cps19_ethnicity_chinese',
       'cps19_ethnicity_dutch', 'cps19_ethnicity_english',
       'cps19_ethnicity_french', 'cps19_ethnicity_french_can',
       'cps19_ethnicity_german', 'cps19_ethnicity_hispanic',
       'cps19_ethnicity_indian', 'cps19_ethnicity_inuk_inuit',
       'cps19_ethnicity_irish', 'cps19_ethnicity_italian',
       'cps19_ethnicity_metis', 'cps19_ethnicity_polish',
       'cps19_ethnicity_quebecois', 'cps19_ethnicity_scottish',
       'cps19_ethnicity_ukranian', 'cps19_sexuality',
       'cps19_language_english', 'cps19_language_french',
       'cps19_language_arabic', 'cps19_language_chinese',
       'cps19_language_filipino', 'cps19_language_german',
       'cps19_language_indian', 'cps19_language_italian',
       'cps19_language_korean', 'cps19_language_pakistani',
       'cps19_language_persian', 'cps19_language_russian',
       'cps19_language_spanish', 'cps19_language_tamil',
       'cps19_language_vietnamese','cps19_language_aborginal', 'cps19_employment',
       'cps19_union', 'cps19_children', 'cps19_income_number', 'cps19_marital',
       'cps19_household'])


In [32]:
y = df['cps19_votechoice']

In [33]:
X.describe()

,cps19_citizenship_Canadian citizen,cps19_yob_1920,cps19_yob_1921,cps19_yob_1922,cps19_yob_1923,cps19_yob_1924,cps19_yob_1925,cps19_yob_1926,cps19_yob_1927,cps19_yob_1928,...,cps19_household_29,cps19_household_33,cps19_household_34,cps19_household_40,cps19_household_54,cps19_household_69,cps19_household_89,cps19_household_501,cps19_household_50000,cps19_household_72000
count,11812.0,11812.000000,11812.000000,11812.000000,11812.000000,11812.000000,11812.000000,11812.000000,11812.000000,11812.000000,...,11812.000000,11812.000000,11812.000000,11812.000000,11812.000000,11812.000000,11812.000000,11812.000000,11812.000000,11812.000000
mean,1.0,0.002032,0.000254,0.000169,0.000085,0.000085,0.000085,0.000169,0.000085,0.000423,...,0.000085,0.000085,0.000085,0.000085,0.000085,0.000085,0.000085,0.000085,0.000085,0.000085
std,0.0,0.045032,0.015935,0.013012,0.009201,0.009201,0.009201,0.013012,0.009201,0.020571,...,0.009201,0.009201,0.009201,0.009201,0.009201,0.009201,0.009201,0.009201,0.009201,0.009201
min,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [34]:
y.value_counts()

Liberal Party         4201
Conservative Party    3767
ndp                   1851
Green Party           1075
Bloc Quebecois         630
People's Party         288
Name: cps19_votechoice, dtype: int64

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=28)
Counter(y_train)

Counter({'Bloc Quebecois': 467,
         'Liberal Party': 3168,
         'ndp': 1371,
         'Conservative Party': 2823,
         'Green Party': 814,
         "People's Party": 216})

In [36]:
Counter(y_test)

Counter({'Liberal Party': 1033,
         'Bloc Quebecois': 163,
         'ndp': 480,
         'Conservative Party': 944,
         'Green Party': 261,
         "People's Party": 72})

In [37]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=28)
rf_model = rf_model.fit(X_train, y_train)

In [38]:
from sklearn.metrics import balanced_accuracy_score
y_pred = rf_model.predict(X_test)
acc_score = balanced_accuracy_score(y_test, y_pred)
acc_score

0.3964423961600232

In [39]:
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Liberal Party", "Conservative Party", "ndp", "Green Party", "Bloc Qu<e9>b<e9>cois", "People's Party"], columns=["Liberal Party", "Conservative Party", "ndp", "Green Party", "Bloc Qu<e9>b<e9>cois", "People's Party"])
cm_df

,Liberal Party,Conservative Party,ndp,Green Party,Bloc Qu<e9>b<e9>cois,People's Party
Liberal Party,64,45,0,50,0,4
Conservative Party,22,824,0,93,0,5
ndp,9,87,5,119,0,41
Green Party,4,54,0,945,0,30
Bloc Qu<e9>b<e9>cois,1,55,0,16,0,0
People's Party,8,137,2,247,0,86


In [40]:
feat_importances = pd.Series(rf_model.feature_importances_, index=X.columns)
feat_importances.sort_values(ascending=False, inplace=True)

In [41]:
print(feat_importances)

cps19_fed_gov_sat_Fairly satisfied        0.025874
cps19_fed_gov_sat_Not at all satisfied    0.025870
cps19_lib_promises_Strongly disagree      0.022610
cps19_snclav_Not at all well              0.017868
cps19_lib_promises_Somewhat agree         0.015401
                                            ...   
cps19_income_number_35600.0               0.000000
cps19_income_number_34592.0               0.000000
cps19_income_number_152500.0              0.000000
cps19_income_number_154000.0              0.000000
cps19_citizenship_Canadian citizen        0.000000
Length: 1634, dtype: float64
